In [1]:
#Key Expansion: key 128 bits / 16 bytes /32 hex

class key_schedule():
    #initialization
    def __init__(self,key_32hex):
        self.key_32hex = key_32hex
        #row and column are transposed
        self.w = [[None for i in range(4)]for i in range(44)]
        
        self.rcon = ['01', '02', '04', '08', '10', '20', '40', '80', '1B', '36']
        self.sbox = [
            ['63', '7c', '77', '7b', 'f2', '6b', '6f', 'c5', '30', '01', '67', '2b', 'fe', 'd7', 'ab', '76'],
            ['ca', '82', 'c9', '7d', 'fa', '59', '47', 'f0', 'ad', 'd4', 'a2', 'af', '9c', 'a4', '72', 'c0'], 
            ['b7', 'fd', '93', '26', '36', '3f', 'f7', 'cc', '34', 'a5', 'e5', 'f1', '71', 'd8', '31', '15'],
            ['04', 'c7', '23', 'c3', '18', '96', '05', '9a', '07', '12', '80', 'e2', 'eb', '27', 'b2', '75'],
            ['09', '83', '2c', '1a', '1b', '6e', '5a', 'a0', '52', '3b', 'd6', 'b3', '29', 'e3', '2f', '84'],
            ['53', 'd1', '00', 'ed', '20', 'fc', 'b1', '5b', '6a', 'cb', 'be', '39', '4a', '4c', '58', 'cf'], 
            ['d0', 'ef', 'aa', 'fb', '43', '4d', '33', '85', '45', 'f9', '02', '7f', '50', '3c', '9f', 'a8'],
            ['51', 'a3', '40', '8f', '92', '9d', '38', 'f5', 'bc', 'b6', 'da', '21', '10', 'ff', 'f3', 'd2'], 
            ['cd', '0c', '13', 'ec', '5f', '97', '44', '17', 'c4', 'a7', '7e', '3d', '64', '5d', '19', '73'],
            ['60', '81', '4f', 'dc', '22', '2a', '90', '88', '46', 'ee', 'b8', '14', 'de', '5e', '0b', 'db'],
            ['e0', '32', '3a', '0a', '49', '06', '24', '5c', 'c2', 'd3', 'ac', '62', '91', '95', 'e4', '79'],
            ['e7', 'c8', '37', '6d', '8d', 'd5', '4e', 'a9', '6c', '56', 'f4', 'ea', '65', '7a', 'ae', '08'],
            ['ba', '78', '25', '2e', '1c', 'a6', 'b4', 'c6', 'e8', 'dd', '74', '1f', '4b', 'bd', '8b', '8a'],
            ['70', '3e', 'b5', '66', '48', '03', 'f6', '0e', '61', '35', '57', 'b9', '86', 'c1', '1d', '9e'],
            ['e1', 'f8', '98', '11', '69', 'd9', '8e', '94', '9b', '1e', '87', 'e9', 'ce', '55', '28', 'df'], 
            ['8c', 'a1', '89', '0d', 'bf', 'e6', '42', '68', '41', '99', '2d', '0f', 'b0', '54', 'bb', '16']

        ]
    
    def bin_to_hex(self,bit):
        bin_to_decimal = int(bit,2)
        decimal_to_hex = hex(bin_to_decimal)[2:].zfill(2)
        return decimal_to_hex.upper()

    def hex_to_bin(self,_hex):
        hex_to_decimal = int(_hex,16)
        decimal_to_bin = bin(hex_to_decimal)[2:].zfill(8)
        return decimal_to_bin

    def XOR(self,hex1,hex2):
        bit1 = self.hex_to_bin(hex1)
        bit2 = self.hex_to_bin(hex2)
        xor_result = ''
        for index in range(len(bit1)):
            r = (int(bit1[index])+int(bit2[index])) % 2
            xor_result = xor_result + str(r) 
        xor_result_hex = self.bin_to_hex(xor_result) 
        return xor_result_hex    
    
    def g_function(self,w,i):
        #for i th round
        left_shift_w = w[1:] + w[:1]
        sub_w = list()
        
        #byte substitution
        for j in left_shift_w:
            sub_w.append(self.sbox[int(j[0],16)][int(j[1],16)])
        
        #adding round constant(only first component)
        sub_first = sub_w[0]
        r_constant = self.rcon[i-1]
        after_xor = self.XOR(sub_first,r_constant)
        sub_w[0] = after_xor
        
        return sub_w
  
    def first_4word(self):
        quotient = int(len(self.key_32hex)/2)
        m,n = 0,0
        for i in range(quotient):
            item = self.key_32hex[2*i] + self.key_32hex[2*i+1]
            
            if None in self.w[n]:
                self.w[n][m] = item 
                m = m + 1
            else:
                self.w[n+1][0] = item
                n,m = n+1,1

    '''key expansion'''
    #for 10 round 
    def key_expansion(self):
        self.first_4word()
        #for i round
        for i in range(1,11):
            #w_4i
            w1 = self.g_function(self.w[4*i-1],i)
            w2 = self.w[4*i-4] 
            for j in range(4):
                self.w[4*i][j] = self.XOR(w1[j],w2[j])
            #w_4i+1
            for k in range(4):
                self.w[4*i+1][k] = self.XOR(self.w[4*i][k],self.w[(4*i+1)-4][k])
            #w_4i+2
            for l in range(4):
                self.w[4*i+2][l] = self.XOR(self.w[4*i+1][l],self.w[(4*i+2)-4][l])
            #w_4i+3
            for m in range(4):
                self.w[4*i+3][m] = self.XOR(self.w[4*i+2][m],self.w[(4*i+3)-4][m])
        
        return self.w  

a = key_schedule('5468617473206D79204B756E67204675')
(a.key_expansion())

[['54', '68', '61', '74'],
 ['73', '20', '6D', '79'],
 ['20', '4B', '75', '6E'],
 ['67', '20', '46', '75'],
 ['E2', '32', 'FC', 'F1'],
 ['91', '12', '91', '88'],
 ['B1', '59', 'E4', 'E6'],
 ['D6', '79', 'A2', '93'],
 ['56', '08', '20', '07'],
 ['C7', '1A', 'B1', '8F'],
 ['76', '43', '55', '69'],
 ['A0', '3A', 'F7', 'FA'],
 ['D2', '60', '0D', 'E7'],
 ['15', '7A', 'BC', '68'],
 ['63', '39', 'E9', '01'],
 ['C3', '03', '1E', 'FB'],
 ['A1', '12', '02', 'C9'],
 ['B4', '68', 'BE', 'A1'],
 ['D7', '51', '57', 'A0'],
 ['14', '52', '49', '5B'],
 ['B1', '29', '3B', '33'],
 ['05', '41', '85', '92'],
 ['D2', '10', 'D2', '32'],
 ['C6', '42', '9B', '69'],
 ['BD', '3D', 'C2', '87'],
 ['B8', '7C', '47', '15'],
 ['6A', '6C', '95', '27'],
 ['AC', '2E', '0E', '4E'],
 ['CC', '96', 'ED', '16'],
 ['74', 'EA', 'AA', '03'],
 ['1E', '86', '3F', '24'],
 ['B2', 'A8', '31', '6A'],
 ['8E', '51', 'EF', '21'],
 ['FA', 'BB', '45', '22'],
 ['E4', '3D', '7A', '06'],
 ['56', '95', '4B', '6C'],
 ['BF', 'E2', 'BF', '90'],
 